In [4]:
conda install -c conda-forge ipython-sql

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda33

  added / updated specs:
    - ipython-sql


The following NEW packages will be INSTALLED:

  ipython-sql        conda-forge/win-64::ipython-sql-0.3.9-py38h32f6830_1002
  prettytable        conda-forge/noarch::prettytable-2.2.1-pyhd8ed1ab_0
  python_abi         conda-forge/win-64::python_abi-3.8-2_cp38
  sqlparse           conda-forge/noarch::sqlparse-0.4.2-pyhd8ed1ab_0

The following packages will be UPDATED:

  conda              pkgs/main::conda-4.10.1-py38haa95532_1 --> conda-forge::conda-4.10.3-py38haa244fe_2


Preparing transaction: ...working... done
Verifying transaction: ...working... failed

Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\Anaconda33




In [7]:
import sqlalchemy

# Introduction

In [9]:
sqlalchemy.create_engine('sqlite:///factbook.db')
%load_ext sql
%sql sqlite:///factbook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [10]:
%reload_ext sql

# Overview Data

In [16]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
   sqlite:///your_database_name.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* `name` — the name of the country.
* `area`— the country's total area (both land and water).
* `area_land` — the country's land area in [square kilometers](https://www.cia.gov/library/publications/the-world-factbook/rankorder/2147rank.html).
* `area_water` — the country's waterarea in square kilometers.
* `population` — the country's population.
* `population_growth` — the country's population growth as a percentage.
* `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
* `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

Let's start by calculating some summary statistics and look for any outlier countries.

# Summary Statistics

In [22]:
%%sql
SELECT MIN(population) AS min_pop, 
       MAX(population) AS max_pop, 
       MIN(population_growth) AS min_pop_growth, 
        MAX(population_growth) AS max_pop_growth
        
FROM facts;

 * sqlite:///factbook.db
   sqlite:///your_database_name.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


In [27]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MIN(population)
                   FROM facts
                   );

 * sqlite:///factbook.db
   sqlite:///your_database_name.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [28]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MAX(population)
                   FROM facts
                   );

 * sqlite:///factbook.db
   sqlite:///your_database_name.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Recompute the summary statistics you found earlier while excluding the row for `the whole world`.

# Summary Statistics Revised, Except for the Whole World

In [29]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
        MAX(population_growth) AS max_pop_growth 
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
   sqlite:///your_database_name.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


# Exploring Average Population and Area

In [30]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
   sqlite:///your_database_name.db
Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


# Search for Densely Populated Countries

Characteristic:
* Above-average values for population.
* Below-average values for area.

In [34]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts
                    WHERE name != 'World'
                   )
AND area < (SELECT AVG(area)
           FROM facts
           WHERE name != 'World'
           );

 * sqlite:///factbook.db
   sqlite:///your_database_name.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31
